In [ ]:
from datetime import datetime
import csv
from pprint import pprint
from DrissionPage import ChromiumPage
import random
import time

In [ ]:
#设置随机休眠时间最大最小值
MIN_SLEEP_TIME = 3
MAX_SLEEP_TIME = 7
#随机休眠，模拟人工操作间隔
def random_sleep():
    sleep_time = random.uniform(MIN_SLEEP_TIME, MAX_SLEEP_TIME)
    print(f"随机休眠 {sleep_time:.2f} 秒...")
    time.sleep(sleep_time)


#使用with语句管理文件资源，自动关闭文件
with open('D:/CityU日常/COM5507/data.csv',mode='w',encoding='UTF-8',newline='') as f:
    #创建字典写入对象，指定表头字段
    csv_writer=csv.DictWriter(f,fieldnames=['昵称','地区','日期','评论'])
    #写入
    csv_writer.writeheader()

    #打开浏览器
    dp=ChromiumPage()
    dp.listen.start('comment/list/')
    random_sleep() #降低请求风险
    #访问网站
    dp.get('https://www.douyin.com/video/7555093612019698986?module=%E7%83%AD%E6%A6%9C') #根据需要切换网址
    random_sleep() #模拟人工操作
    
    
    #构建循环翻页，区间是1到79页，尽量可以充分翻完，数值可进行调整
    for page in range (1,80):
        print(f'正在采集{page}页的数据内容')
        #等待数据包加载
        resp=dp.listen.wait()

        #获取响应数据
        try:
            json_data=resp.response.body
            #解析数据，获取评论信息所在的列表
            try:
                comments=json_data['comments']
            except KeyError as e:
                print(f'获取评论列表出现异常，异常信息：{e}')
                comments=[]
            #遍历评论列表，提取每条评论具体数据信息
            for index in comments:
                try:
                    create_time=index['create_time']
                    date=str(datetime.fromtimestamp(create_time))
                    #尝试从ip_label获取地区信息
                    try:
                        region=index['ip_label']
                    except KeyError:
                        ip_client_info=index.get('client_info',{})
                        region=ip_client_info.get('province','未知') if ip_client_info else "未知"
                except KeyError as e:
                    print(f"处理单个评论数据出现异常，异常信息：{e}，当前评论数据：{index}")
                    continue
                dit={
                    '昵称':index['user']['nickname'],
                    '地区':region,
                    '日期':date,
                    '评论':index['text']
                }
                try:
                    csv_writer.writerow(dit)
                    print(dit)
                except Exception as e:
                    print(f"写入CSV文件出现异常,异常信息：{e}，当前数据：{dit}")

                
                #查找下一页元素，判断是否存在
                next_page=dp.ele("css:.Rcc71LyU")
                if next_page:
                    dp.scroll.to_see(next_page)
                else:
                    print("未找到下一页元素，可能已到达最后一页或者页面结构有变化")
        except Exception as e:
            print(f"整体页面数据处理出现异常，异常信息：{e}")


正在采集1页的数据内容
{'昵称': '是我林北呀', '地区': '湖北', '日期': '2025-11-06 15:37:37', '评论': '女主太美了吧，她叫什么呀'}
{'昵称': 'An', '地区': '广东', '日期': '2025-10-08 02:08:27', '评论': '这个皇帝昏迷意识到狗身上的设定很像宫斗不如养条狗，但是又搞个重生的事，又不太一样了。\r原著女主很聪慧，知道自己家功高盖主，自己只是皇帝钳制家人的棋子，故意装作嚣张跋扈，不通文墨。而且女主也知道皇帝有白月光，拿她当白月光挡箭牌，宫中妃嫔的陷害都落女主身上。皇帝昏迷后，他的白月光让皇帝替身去秽乱后宫，女主也是一接触替身就发现了，立刻吃药装作月事来了，然后让人去调查。\r而皇帝意识一直在狗身上，期间了解到女主的聪慧和很多优点，又发现白月光虚伪的真面目，爱上了女主，直到狗狗的身体被害死，才回到自己身上的。\r这部剧要么只是设定雷同，要么就是魔改了很多。[尬笑]'}
{'昵称': '程程（南京拽妃）', '地区': '江苏', '日期': '2025-11-05 13:36:41', '评论': '[捂脸][捂脸][捂脸]面相[赞]很绝'}
{'昵称': '锦悠屿铄', '地区': '江西', '日期': '2025-11-05 13:00:42', '评论': '已经严格到短剧看不了男主不洁的了，但是女主貌美又想停下来再看看[微笑]'}
{'昵称': '昏戳戳', '地区': '安徽', '日期': '2025-10-17 21:06:29', '评论': '这部剧的剧情设计确实令人惊喜。开篇看似平淡的日常场景，却在不经意间埋下诸多伏笔，随着情节推进，各条故事线逐渐交织，形成一张紧密的叙事网络。主角的成长弧光清晰可见，从最初的懵懂青涩到后来的沉稳坚韧，每个阶段的转变都自然且有说服力。配角们也各具特色，他们的故事不仅丰富了剧情层次，还与主线相辅相成，共同构建出一个真实可感的世界。反转情节的设置恰到好处，既在意料之外又在情理之中，总能让观众在恍然大悟时，对之前的细节产生新的理解，这种层层递进的叙事方式让人沉浸其中，忍不住想要一直追下去。'}
正在采集2页的数据内容
{'昵称': '是我林北呀', '地区': '湖北', '日期': '2025-11-06 15:37:37', 